In [1]:
import pandas as pd
import numpy as np
import os
import re
import unicodedata as ud


In [2]:
## read data in

In [3]:
def split_coor_basin(basin_file):
    ''' split coordinate for each basin row data
    '''
    basin = pd.read_excel(basin_file)
    basin.set_index('Unnamed: 0',inplace=True)
    basin_row_con = {} ## data container
#     basin_row_2d = [] ## basin row container
    #iterate rows
    for i in range(1,len(basin)):
        basin_row_list = [] # row data container
        basin_row = basin.iloc[i]
        for j in range(len(basin_row)):
            basin_row_pari = []
            if ',' in str(basin_row[j]) and basin_row[j]:
                basin_row_col_split = str(basin_row[j]).split(',') #split data
                if re.match(r'-*\d',basin_row_col_split[0]) and re.match(r'-*\d',basin_row_col_split[1]): ##data is not null and valid
                    basin_row_pari.append(basin_row_col_split[0]) #saving long
                    basin_row_pari.append(basin_row_col_split[1]) #saving lat
            basin_row_list.append(basin_row_pari) #saving pair to basin_row
        basin_row_con[basin_row.name] = basin_row_list ## append list to 2d row container
#         basin_row_con[basin_row.name] = basin_row_2d
    print('split basin row data')
    return basin_row_con

In [5]:
split_basin_data = split_coor_basin('basin.xlsx')

split basin row data


In [321]:
# split_basin_data['Faroe - Shetland Escarpment']

[['-4.695346923999978', '60.08393867900003'],
 ['-4.663409999999942', '60.11163000000005'],
 ['-4.629279999999937', '60.13972000000006'],
 ['-4.595069999999964', '60.17101000000007'],
 ['-4.500299999999925', '60.27301000000006'],
 ['-4.462619999999959', '60.31861000000004'],
 ['-4.392039999999952', '60.40991000000008'],
 ['-4.369029999999952', '60.44142000000005'],
 ['-4.311439999999948', '60.52338000000003'],
 ['-4.286729999999977', '60.55965000000003'],
 ['-4.250459999999975', '60.61488000000003'],
 ['-4.230579999999975', '60.64965000000009'],
 ['-4.179539999999974', '60.72375000000006'],
 ['-4.120219999999961', '60.81208000000004'],
 ['-4.098739999999964', '60.84682000000004'],
 ['-4.067539999999951', '60.88775000000004'],
 ['-4.034779999999955', '60.92706000000004'],
 ['-3.966159999999945', '61.00081000000004'],
 ['-3.926969999999926', '61.04159000000005'],
 ['-3.865019999999959', '61.10109000000005'],
 ['-3.824299999999937', '61.13863000000004'],
 ['-3.785329999999931', '61.168220

In [6]:
def gen_basin_df(split_basin_data):
    '''generate dataframe for each field with such column,basin,lat,and long
    
    '''
#     basin_tuple = {}
    basin_dfs = []
    for key in split_basin_data.keys(): ##iterate through the libaries
        basin_df = pd.DataFrame.from_dict(split_basin_data[key]) ## gen dataframe
        basin_df = basin_df[~basin_df.isnull()]
        basin_df.dropna(inplace=True)
        basin_df['basin'] = key
        basin_dfs.append(basin_df) ## append dataframe to list
        
    
    basin_df_con = pd.concat(basin_dfs) ##concata together
#     basin_df_con = basin_df_con[~basin_df_con.isnull()] ##find null value
#     basin_df_con.dropna(inplace=True)
    basin_df_con.columns = ['long','lat','basin']
    basin_df_con['long'] = basin_df_con['long'].map(float)
    basin_df_con['lat'] = basin_df_con['lat'].map(float)
    #set index
    basin_df_con.set_index('basin',inplace=True)
    
    print('get basin dataframe in a clean form')
    return basin_df_con

In [7]:
basin_dfs = gen_basin_df(split_basin_data)

get basin dataframe in a clean form


In [8]:
basin_dfs.

,long,lat
basin,,
Faroe - Shetland Escarpment,-4.695347,60.083939
Faroe - Shetland Escarpment,-4.663410,60.111630
Faroe - Shetland Escarpment,-4.629280,60.139720
Faroe - Shetland Escarpment,-4.595070,60.171010
Faroe - Shetland Escarpment,-4.500300,60.273010


In [11]:
def get_basin_dict(basins_df):
    '''convert basin row data to list of tuple
    '''
    basins_dict = {}
    basins = basin_dfs.index.unique()
    for basin in basins:
         
        basin_ready_tuple = basin_dfs[basin_dfs.index == basin] ##basin index equal to one of element
        basin_tuples = [tuple(x) for x in basin_ready_tuple.values] ## convert data frame to tuple
        basins_dict[basin] = basin_tuples
        
    print('convert to basin dict')
    return basins_dict

In [13]:
basin_tuple = get_basin_dict(basin_dfs)

convert to basin dict


In [400]:
def check_inside_poly(field_coor,basin_tuple,basin):
    '''field coordinate and one basin_tuple
    Args:
        field coordinate,(1,2),tuple
        basin tuple,[(1,2),(3,4).......],list
        basin, basin name,string
    Return:
        inside or not and basin name,(True,'Basin_name')
    '''
    inside_rect = False #inside flag
    crossing_point = 0
    max_long = max([x[0] for x in basin_tuple]) #max long
    min_long = min([x[0] for x in basin_tuple]) #min long
    max_lat = max([x[1] for x in basin_tuple]) #max lat
    min_lat = min([x[1] for x in basin_tuple]) #min lat
    if ((min_long <= field_coor[0] and max_long>=field_coor[1])
        and (min_lat < field_coor[1] and max_lat > field_coor[1])): # point inside the rectangele
        for i in range(len(basin_tuple)-1): #iterate the poly points
            slope = (basin_tuple[i+1][1] - basin_tuple[i][1])/ (basin_tuple[i+1][0] - basin_tuple[i][0])
            cond1 = (basin_tuple[i][0] <= field_coor[0]) and  (field_coor[0] < basin_tuple[i+1][0])
            cond2 = (basin_tuple[i+1][0] <= field_coor[0]) and  (field_coor[0] < basin_tuple[i][0])            
            test_slope = (field_coor[1] - basin_tuple[i][1])/(field_coor[0]- basin_tuple[i][0])
            above = float(test_slope) <= float(slope)
#             print(type(test_slope),test_slope)
#             print(slope,type(slope))
            if (cond1 or cond2) and above:
                crossing_point += 1
        if crossing_point %2 == 0:
            inside_rect = True
            
    return inside_rect,basin
    
    
    
    

In [18]:
def determin_range(field_file,basin_tuple):
    '''project field lat and long to correspoding basin name
    
    Args:
        field,dataframe with storing field coordinates
        basin,basin coordinates
    Returns:
        field-basin, corresponding dataframe
        
    '''
    ## read in field data
    field = pd.read_excel(field_file)
#     basin = pd.read_excel('basin.xlsx')
#     field_basin = {}
    field_arr = field[['field','long','lat']]
    field_arr.set_index('field',inplace=True)
    field_arr['basin'] = None
    for i in range(len(field_arr)):
        field_row = field_arr.iloc[i] ## get field coord
        field_coor = tuple(field_row.to_list())[0:2] ## get field coordinate tuple
#         print(field_coor,type(field_coor))
#         break
        ## for each point searching inside basins
        for key in basin_tuple.keys(): ## for one basin
            ##check for each basin coorinates
            is_inside,basin_name = check_inside_poly(field_coor,basin_tuple[key],key) ##match field to basin
            if is_inside: ## if point inside this basin
                field_arr['basin'] = basin_name ## save basin name to field row
                
    field_arr.to_excel('field_basin_coor.xlsx')

In [392]:
for i in range(len(basin_tuple)-1): #iterate the poly points
    slope = (basin_tuple_co[i+1][1] - basin_tuple_co[i][1])/ (basin_tuple_co[i+1][0] - basin_tuple_co[i][0])
    print(slope,type(slope))
    slope = float(slope)
    if 0.2> slope:
        pass
    break

0.867062870550383 <class 'numpy.float64'>


In [ ]:
if __name__ == '__main__':
    field_path = 'field.xlsx'
    basin_path = 'basin.xlsx'
     #split basin row data to basin data in dict from,{'basin':[[long,lat].....]}
    split_basin_row = split_coor_basin(basin_path)
    ## get basins df together and drop all the null value in form of,basin,long,lat
    basins_df = gen_basin_df(split_basin_row)
    ## convert basins df to dict form in such a form, {'basin':[[(long,lat)......]]}
    basins_dict = get_basin_dict(basins_df)
    ## determinde one point belong or not to a basin and write result to xlsx 
    determin_range(field_path,basins_dict)
    
    